For this project, you will create a machine learning model to predict the stage of cancer (I, II, III,
or IV) from both RNA and protein-level gene expression for clear cell renal cell carcinoma
(CCRCC) in CPTAC. Stage of cancer can be found using the tumor_stage_pathological column
within the CPTAC clinical data. You can access the data the exact same way as BRCA,
substituting the accession code

In [1]:
import cptac
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

1) Select what features to include in the model by finding the top 5 most differentially
expressed proteins between Stage I and Stage III patients in CPTAC protein data. Repeat
this process to find the top 5 most differential expression RNA between Stage I and Stage
III patients in the CPTAC RNA data.
a) Use tumor_stage_pathological in the CPTAC clinical data.


In [2]:
# cptac.download('Ccrcc') # shouldn't need this if already downloaded
ccrcc = cptac.Ccrcc()

In [3]:
clinical = ccrcc.get_clinical()
rna = ccrcc.get_transcriptomics()
protein = ccrcc.get_proteomics()
protein.columns = protein.columns.get_level_values(0)

In [6]:
masked_clin = clinical.dropna(subset=['tumor_stage_pathological'])

nan_mask_rna = rna.index.isin(masked_clin.index)
nan_mask_protein = protein.index.isin(masked_clin.index)

masked_protein = protein[nan_mask_protein]
masked_rna = rna[nan_mask_rna]

shape before dropping na: (194, 171)
shape after dropping na: (110, 171)
shape of rna: (110, 19275)
shape of protein: (110, 11710)


In [11]:
stageI_mask = (masked_clin['tumor_stage_pathological']=='Stage I')
stageIII_mask = (masked_clin['tumor_stage_pathological']=='Stage III')
stageI_clin = masked_clin[stageI_mask]
stageIII_clin = masked_clin[stageIII_mask]

print(f'shape of stage1_clin: {stageI_clin.shape}. and stageIII_clin: {stageIII_clin.shape}')

shape of stage1_clin: (52, 171). and stageIII_clin: (33, 171)


In [169]:
protein_dropped = masked_protein.dropna(axis=1)
print(protein_dropped.shape)

(110, 6668)


In [170]:
stageI_protein = protein_dropped[stageI_mask]

stageIII_protein = protein_dropped[stageIII_mask]

shape of stage1_rna: (52, 19275). and stageIII_rna: (33, 19275)
shape of stage1_protein: (52, 6668). and stageIII_protein: (33, 6668)


In [217]:
#Going to log scale all values in masked_rna, then divide into stage1 and stage3 dataframes

def log_scale_df(df):
    
    for row in range(df.shape[0]):
        for col in range(df.shape[1]):
            val = df.iloc[row,col]
            if val>0:
                df.iloc[row,col] = np.log2(val)
            elif val<0:
                df.iloc[row,col] = -np.log(np.abs(val))
            else:
                df.iloc[row,col] = 0
    
    return df 

In [218]:
log_masked_rna = log_scale_df(masked_rna)  

In [219]:
stageI_rna = log_masked_rna[stageI_mask]
stageIII_rna = log_masked_rna[stageIII_mask]

In [232]:
stageI_rna_means = np.array([stageI_rna[gene].mean() for gene in stageI_rna.columns])
stageIII_rna_means = np.array([stageIII_rna[gene].mean() for gene in stageIII_rna.columns])

In [236]:
diff_arr_rna = np.absolute(stageI_rna_means - stageIII_rna_means)

top_5_diff_rna = [0]*5

for diff in diff_arr_rna:
    for i in range(5):
        if diff > top_5_diff_rna[i]:
            top_5_diff_rna[i] = diff
            break
print(top_5_diff_rna)          
            
top_5_indices_rna = [list(diff_arr_rna).index(i) for i in top_5_diff_rna]
print(top_5_indices_rna)

top_5_de_rna = [log_masked_rna.columns[i] for i in top_5_indices_rna]
print(top_5_de_rna)

[2.1381042943199633, 1.921327839274935, 1.8684131376813253, 1.843991616868239, 1.6923448537989814]
[295, 17898, 14838, 17085, 14855]
['ADAMTS20', 'UNC5D', 'SHCBP1L', 'TMPRSS7', 'SHOX']


In [171]:
stageI_protein_means = np.array([stageI_protein[protein].mean() for protein in stageI_protein.columns])
stageIII_protein_means = np.array([stageIII_protein[protein].mean() for protein in stageIII_protein.columns])

In [235]:
diff_arr_protein = np.absolute(stageI_protein_means - stageIII_protein_means)

filtered_diff_arr = [element if not isinstance(element, pd.Series) else np.nan for element in diff_arr_protein]

top_5_diff_protein = [0]*5

for diff in filtered_diff_arr:
    for i in range(5):
        if diff > top_5_diff_protein[i]:
            top_5_diff_protein[i] = diff
            break
print(top_5_diff_protein)
            
top_5_indices_protein = [list(filtered_diff_arr).index(i) for i in top_5_diff_protein]
print(top_5_indices_protein)

top_5_de_protein = [protein_dropped.columns[i] for i in top_5_indices_protein]
print(top_5_de_protein)

[0.8621652797318512, 0.6034531374509484, 0.5573026444527898, 0.5317585338534563, 0.5194684913126811]
[2139, 2480, 2475, 2476, 5609]
['FTL', 'HBZ', 'HBB', 'HBD', 'SPTA1']


In [238]:
# now want to create a separate dataframe which contains the patients, their expression for the top5 de genes and proteins.
# and include their tumor pathological stage

classification_df = pd.DataFrame()

for patient in masked_clin.index:
    patient_data = {}
    
    rnas = ['ADAMTS20', 'UNC5D', 'SHCBP1L', 'TMPRSS7', 'SHOX']
    for rna in rnas:
        patient_data[rna] = masked_rna.loc[patient,rna]
    
    proteins = ['FTL', 'HBZ', 'HBB', 'HBD', 'SPTA1']
    for protein in proteins:
        patient_data[protein] = masked_protein.loc[patient,protein]

    patient_data['tumor_stage_pathological'] = masked_clin.loc[patient,'tumor_stage_pathological']
    
    patient_data = pd.DataFrame(patient_data,index=[0])

    classification_df = pd.concat([classification_df,patient_data],axis=0,ignore_index=True)

In [240]:
classification_df

,ADAMTS20,UNC5D,SHCBP1L,TMPRSS7,SHOX,FTL,HBZ,HBB,HBD,SPTA1,tumor_stage_pathological
0,-6.674653,-3.637167,0.000000,0.000000,-7.109794,-1.724339,-0.370098,-0.762287,-0.627764,-0.643136,Stage III
1,-5.307190,1.026229,0.000000,0.000000,-7.549687,-0.363228,0.240576,-0.034944,0.042620,-0.171893,Stage I
2,1.993335,-7.541405,-6.509080,0.000000,0.000000,-0.977364,-0.087641,-0.371128,-0.329051,-0.345668,Stage IV
3,-3.212043,1.753551,0.000000,-5.736581,0.000000,1.301930,0.981930,0.855097,0.642599,0.734517,Stage I
4,-6.807089,-1.388781,0.000000,0.000000,0.000000,-1.496648,-0.441854,-0.939448,-0.942320,-0.977896,Stage III
5,-3.543537,2.281417,-5.380616,0.000000,-7.300606,-1.895355,1.055353,0.769956,0.769616,0.794621,Stage III
6,0.000000,-3.903285,0.000000,0.000000,0.000000,-1.280847,-0.583183,-0.312478,-0.242648,-0.114342,Stage IV
7,-4.506756,-3.852598,0.000000,-5.598335,-3.563386,0.602369,0.172828,0.532612,0.657434,0.651421,Stage I
8,-3.385180,-3.262805,-6.359763,-4.292335,0.000000,-1.919103,-0.724898,-1.038594,-1.017786,-0.901090,Stage III
9,-5.983276,1.961101,-4.498428,0.000000,0.000000,1.109694,0.919934,1.651025,1.613681,1.487945,Stage III


In [256]:
X = classification_df.drop('tumor_stage_pathological',axis=1)
y = classification_df['tumor_stage_pathological']

In [267]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier # default number of neighbors looked at is 5
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

In [265]:
scaler = StandardScaler()
scaled_X = scaler.fit_transform(X)

encoder = OrdinalEncoder()
encoded_y = encoder.fit_transform(y.to_numpy().reshape(-1,1))

In [286]:
classifiers = [KNeighborsClassifier, DecisionTreeClassifier, MLPClassifier, GaussianNB]
classifier_names = ['KNeighborsClassifier', 'DecisionTreeClassifier', 'MLPClassifier', 'GaussianNB']
accuracies = {}
for classifier in classifiers:
   accuracies[classifier] = []


for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, encoded_y, train_size=0.7,stratify=encoded_y)
    
    for classifier in classifiers:
        model = classifier()
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        
        accuracy = sum(y_pred == y_test)/len(y_test)
        
        accuracies[classifier].append(accuracy)
i = 0
for classifier in classifiers:
    avg_accuracy = np.mean(accuracies[classifier])
    print(f'The average accuracy of the {classifier_names[i]} is {str(avg_accuracy*100)[:4]}%.')
    i+=1

The average accuracy of the KNeighborsClassifier is 41.3%.
The average accuracy of the DecisionTreeClassifier is 32.6%.
The average accuracy of the MLPClassifier is 39.2%.
The average accuracy of the GaussianNB is 34.4%.


The KNeighbors Classifier has the best performance, compared to the other classifiers.